# PWC Scraper

This scraper collects texts from the "Artificial Intelligence" subsection of PWC.com: https://www.pwc.com/gx/en/issues/data-and-analytics/artificial-intelligence.html

## Libraries

Loading libraries...

In [5]:
import scrapy
import requests
from scrapy import Selector

import os
import sys
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

import re

import time

## URL List
The scraper collects urls from the insights on the "artificial intelligence" subsection of pwc.com: https://www.pwc.com/gx/en/issues/data-and-analytics/artificial-intelligence.html

In [22]:
start_url = 'https://www.pwc.com/gx/en/issues/data-and-analytics/artificial-intelligence.html'

In [23]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options

chromeOptions = Options()
chromeOptions.add_argument("--kiosk")

driver = webdriver.Chrome(executable_path = 'C:\\bin\\chromedriver', options=chromeOptions)

driver.get(start_url)
time.sleep(2)
driver.find_element_by_css_selector('button#acceptCookie.btn.cookie-accept').click()
time.sleep(2)

y_coord = 4000
coordinates = {'x': 0, 'y': y_coord}
driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
time.sleep(3)

while True:
    try:
        driver.find_element_by_css_selector("a.btn.btn--transparent.collection__load-more.ng-binding.ng-scope").click()
        time.sleep(3)
        y_coord += 1000
        coordinates = {'x': 0, 'y': y_coord}
        driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
        time.sleep(3)
    except (NoSuchElementException, WebDriverException, ElementNotInteractableException) as e:
        print("No more to load")
        break

pageSource = driver.page_source

driver.quit()

No more to load


In [24]:
sel = Selector(text = pageSource) #create Selector object
urldict = {}
   
urldict['titles'] = sel.css('a.collection__item-link > div.collection__item-content > h4.regular.collection__item-heading > span::text').extract()
urldict['urls'] = sel.css('a.collection__item-link::attr(href)').extract()

In [25]:
print(len(urldict['titles']),
     len(urldict['urls']))

100 100


In [26]:
urllist = urldict['urls']

### Save URLs

In [27]:
#URL dump
with open('../data_raw/urls/pwc_urls.txt', 'w') as f:
    for url in urllist:
        f.write(url + "\n")

### Load URLs

In [1]:
# Load URLs
urllist = []
with open('../data_raw/urls/pwc_urls.txt', 'r') as f:
    for line in f:
        urllist.append(line.strip())

print(len(urllist),
      '\n',
     urllist[0])

100 
 https://www.pwc.co.uk/services/sustainability-climate-change/insights/new-nature-economy-nature-risk-rising.html


## Parsing single page

In [2]:
urllist[0:10]

['https://www.pwc.co.uk/services/sustainability-climate-change/insights/new-nature-economy-nature-risk-rising.html',
 'https://www.pwc.co.uk/industries/financial-services/insights/ai-in-financial-services.html',
 'https://www.pwc.ch/en/insights/digital/governance-is-the-foundation-of-responsible-ai.html',
 'https://www.pwc.co.uk/ghost/if-ai-isnt-responsible-it-isnt-truly-intelligent.html',
 'https://www.pwc.ch/en/insights/digital/if-ai-isnt-responsible-it-isnt-truly-intelligent.html',
 'https://www.pwc.co.uk/ghost/blogs/the-role-of-ai-achieving-global-sustainable-development-goals.html',
 'https://www.pwc.co.uk/ghost/cfo-artificial-intelligence.html',
 'https://www.pwc.com/gx/en/issues/data-and-analytics/artificial-intelligence/cfo-artificial-intelligence.html',
 'https://www.pwc.com/us/en/services/forensics/library/machine-learning-risk-compliance.html',
 'https://www.pwc.co.uk/who-we-are/corporate-sustainability/assets/documents/pwc-uk-responsible-technology-policy-july-2019.pdf']

In [10]:
page_url = urllist[24]
page_html = requests.get(page_url).content
page_sel = Selector(text = page_html)
page_url

'https://www.pwc.com/gx/en/industries/tmt/publications/global-tmt-semiconductor-report-2019.html'

In [11]:
page_dict = dict()
main_dict = dict()

In [12]:
func_regex = r'\$\(.+\(\).*((\s*|\r*|\n*).*)+'
viewmore_regex = r'Explore our other predictions ((\s+|\r+|\n+|\t+)+)+((\s*|\r*|\n*).*)+'
cont_regex = r'\.pinned.*((\s*|\r*|\n*).*)+'

siteno = 1

page_id = 'site' + str(siteno)

page_text = page_sel.css('section#article > div > div >div > div > div.text-component ::text').extract() #Insights
page_text.append(' '.join(page_sel.css('div#main-body ::text').extract())) #blogs? - 5
page_text.append(' '.join(page_sel.css('div#wrapper > div.centerPar ::text').extract()))
page_text = ' '.join(page_text)

page_text = re.sub(func_regex, '', page_text)
page_text = re.sub(viewmore_regex, '', page_text)
page_text = re.sub(cont_regex, '', page_text)

page_title = page_sel.css('title ::text').extract_first()
page_date = page_sel.xpath('//meta[contains(@name,"pwcReleaseDate")]/@content').extract()
page_type = ''
page_dldate = date.today().strftime('%Y/%m/%d')

page_dict['url'] = page_url
page_dict['title'] = page_title
page_dict['article date'] = page_date
page_dict['download date'] = page_dldate
page_dict['text type'] = page_type
page_dict['text'] = page_text

main_dict[page_id] = page_dict

In [13]:
print(len(main_dict),
      main_dict['site1'])

1 {'url': 'https://www.pwc.com/gx/en/industries/tmt/publications/global-tmt-semiconductor-report-2019.html', 'title': 'Opportunities for the global semiconductor market: PwC\r\n', 'article date': ['2019-03-27T19:46:00.000+01:00'], 'download date': '2020/03/04', 'text type': '', 'text': ' \r\n    \r\n    \r\n    \r\n        \r\n    \r\n\r\n    \r\n    \r\n        \r\n     \r\n\r\n                \r\n          \r\n             \r\n    \r\n\t\t\r\n     \r\n    \r\n             \r\n\r\n                 \r\n                     \r\n                           \r\n                               \r\n                           Opportunities for the global semiconductor market \r\n                           Growing market share by embracing Artificial Intelligence (AI) \r\n                          \r\n                                   \r\n                                       \r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n \r\n \r\n\r\n \r\n\r\n\r\n       \r\n           \r\n               \r\n          

## PWC Spider

In [14]:
from scrapy.crawler import CrawlerProcess

# Create class
class pwc_spider (scrapy.Spider):
    name = "pwc_spider"
    
    def start_requests(self):
        urls = urllist
        
        for url in urls:
            yield scrapy.Request(url = url, callback = self.parse)
            
    #Parsing
    def parse(self, response):
        print(response.url)
                
        page_dict = dict()
        siteno = len(main_dict) + 1
        
        page_url = response.url
        
        page_html = requests.get(page_url).content
        page_sel = Selector(text = page_html)
        
        func_regex = r'\$\(.+\(\).*((\s*|\r*|\n*).*)+'
        viewmore_regex = r'Explore our other predictions ((\s+|\r+|\n+|\t+)+)+((\s*|\r*|\n*).*)+'
        cont_regex = r'\.pinned.*((\s*|\r*|\n*).*)+'

        page_id = 'site' + str(siteno)

        page_text = page_sel.css('section#article > div > div >div > div > div.text-component ::text').extract() #Insights
        page_text.append(' '.join(page_sel.css('div#main-body ::text').extract())) #blogs? - 5
        page_text.append(' '.join(page_sel.css('div#wrapper > div.centerPar ::text').extract()))
        page_text = ' '.join(page_text)

        page_text = re.sub(func_regex, '', page_text)
        page_text = re.sub(viewmore_regex, '', page_text)
        page_text = re.sub(cont_regex, '', page_text)

        page_title = page_sel.css('title ::text').extract_first()
        page_date = page_sel.xpath('//meta[contains(@name,"pwcReleaseDate")]/@content').extract_first()
        page_type = ''
        page_dldate = date.today().strftime('%Y/%m/%d')

        page_dict['url'] = page_url
        page_dict['title'] = page_title
        page_dict['article date'] = page_date
        page_dict['download date'] = page_dldate
        page_dict['text type'] = page_type
        page_dict['text'] = page_text
        page_dict['agency'] = "PwC"

        main_dict[page_id] = page_dict
        
# Initiatlize dictionary
main_dict = dict()

#Run spider
process = CrawlerProcess()
siteno = 0
process.crawl(pwc_spider)
process.start()

2020-03-04 13:20:45 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2020-03-04 13:20:45 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Windows-10-10.0.16299-SP0
2020-03-04 13:20:45 [scrapy.crawler] INFO: Overridden settings: {}
2020-03-04 13:20:45 [scrapy.extensions.telnet] INFO: Telnet Password: 763858fabe9254cf
2020-03-04 13:20:45 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-03-04 13:20:45 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defau

https://www.pwc.co.uk/who-we-are/corporate-sustainability/assets/documents/pwc-uk-responsible-technology-policy-july-2019.pdf


2020-03-04 13:20:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.pwc.co.uk/services/sustainability-climate-change/insights/new-nature-economy-nature-risk-rising.html> (referer: None)
2020-03-04 13:20:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.pwc.co.uk/industries/financial-services/insights/ai-in-financial-services.html> (referer: None)
2020-03-04 13:20:46 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (307) to <GET https://www.pwc.com/gx/en/issues/data-and-analytics/artificial-intelligence/cfo-artificial-intelligence.html> from <GET https://www.pwc.co.uk/ghost/cfo-artificial-intelligence.html>
2020-03-04 13:20:46 [scrapy.dupefilters] DEBUG: Filtered duplicate request: <GET https://www.pwc.com/gx/en/issues/data-and-analytics/artificial-intelligence/cfo-artificial-intelligence.html> - no more duplicates will be shown (see DUPEFILTER_DEBUG to show all duplicates)
2020-03-04 13:20:46 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (3

https://www.pwc.co.uk/services/sustainability-climate-change/insights/new-nature-economy-nature-risk-rising.html


2020-03-04 13:20:46 [urllib3.connectionpool] DEBUG: https://www.pwc.co.uk:443 "GET /services/sustainability-climate-change/insights/new-nature-economy-nature-risk-rising.html HTTP/1.1" 200 24466
2020-03-04 13:20:46 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.pwc.co.uk:443


...

2020-03-04 13:21:34 [urllib3.connectionpool] DEBUG: https://pwc.blogs.com:443 "GET /deals/2017/12/the-uk-fibre-opportunity-six-commercial-considerations-for-investors-.html HTTP/1.1" 200 None
2020-03-04 13:21:34 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): pwc.blogs.com:443


https://pwc.blogs.com/ceoinsights/2018/01/can-emerging-tech-help-ease-ceo-anxiety.html


2020-03-04 13:21:35 [urllib3.connectionpool] DEBUG: https://pwc.blogs.com:443 "GET /ceoinsights/2018/01/can-emerging-tech-help-ease-ceo-anxiety.html HTTP/1.1" 200 None
2020-03-04 13:21:35 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): pwc.blogs.com:443


https://pwc.blogs.com/ceoinsights/2018/01/a-strong-business-outlook-means-its-time-to-invest-in-people-and-in-digital-findings-from-pwcs-2018-.html


2020-03-04 13:21:35 [urllib3.connectionpool] DEBUG: https://pwc.blogs.com:443 "GET /ceoinsights/2018/01/a-strong-business-outlook-means-its-time-to-invest-in-people-and-in-digital-findings-from-pwcs-2018-.html HTTP/1.1" 200 None
2020-03-04 13:21:36 [scrapy.core.engine] INFO: Closing spider (finished)
2020-03-04 13:21:36 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 75146,
 'downloader/request_count': 153,
 'downloader/request_method_count/GET': 153,
 'downloader/response_bytes': 11438274,
 'downloader/response_count': 153,
 'downloader/response_status_count/200': 88,
 'downloader/response_status_count/301': 20,
 'downloader/response_status_count/302': 2,
 'downloader/response_status_count/307': 41,
 'downloader/response_status_count/404': 2,
 'dupefilter/filtered': 13,
 'elapsed_time_seconds': 50.504398,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2020, 3, 4, 12, 21, 36, 10112),
 'httperror/response_ignored_count': 2,
 'httperror/

In [15]:
process.stop()

<DeferredList at 0x2c08b0a1788 current result: []>

In [16]:
len(main_dict)

85

In [17]:
import json

with open('..\\data_raw\\articles\\pwc_articles.json', 'w') as file:
    json.dump(main_dict, file)